In [0]:
from google.colab import files

In [0]:
files.upload()

In [0]:
import pandas as pd;
import numpy as np;
data=pd.read_csv("clusterincluster.txt",sep=" ",header=None)


In [0]:
X=data.iloc[ :, :-1].values
Y=data.iloc[:,2].values


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,test_size=0.2)

In [0]:
#FEATURE_SCALING
from sklearn.preprocessing import StandardScaler
sc_X=StandardScaler()
X_train=sc_X.fit_transform(X_train)
X_test=sc_X.fit_transform(X_test)


In [0]:
#defining the sizes of layer
n_h=4; #number of hidden layer is four
n_inp=X.shape[1];
n_out=1;



In [0]:
def sigmoid(z):
  k=1/(1+np.exp(-z));
  return k;

In [0]:
#initialising the parameters of a NN
def initialisation(n_inp,n_h,n_out):
  W1=np.random.randn(n_h,n_inp)*0.01;
  b1=np.zeros((n_h,1));
  W2=np.random.randn(n_out,n_h)*0.01;
  b2=np.zeros((n_out,1));
  parameters={"W1":W1,
              "b1":b1,
              "W2":W2,
              "b2":b2}
  return parameters;

In [0]:
def forward_prop(X,parameters):
  W1=parameters['W1'];
  W2=parameters['W2'];
  b1=parameters['b1'];
  b2=parameters['b2'];
  #the activation function for the first function can be selected which suits the dataset in the best way.
  
  Z1=np.dot(W1,X)+b1;
  A1=np.tanh(Z1)
  #the activation function for the final layer should be a sigmoid in the case of a classification problem.
  Z2=np.dot(W2,A1)+b2;
  A2=sigmoid(Z2);
  cache={"Z1":Z1,
         "A1":A1,
         "A2":A2,
         "Z2":Z2}
  return A2,cache;

In [0]:
def compute_cost(A2,Y):
  m=Y.shape[0];
  cost=-1/m*(np.multiply(Y,np.log(A2))+np.multiply(1-Y,np.log(1-A2)));
  return cost;

In [0]:
def backward_prop(parameters,cache,X,Y):
  W1 = parameters['W1'];
  W2 = parameters['W2'];
  b1 = parameters['b1'];
  b2 = parameters['b2']; 
  A1 = cache['A1'];
  A2 = cache['A2'];
  Z1=  cache['Z1'];
  Z2=  cache['Z2'];
  m=X.shape[1];
  dZ2=A2-Y;
  dW2=1/m*np.dot(dZ2,A1.T);
  db2=1/m*np.sum(dZ2,axis=1,keepdims=True);
  #here it depends on the activation function of the hidden layer!
  dZ1=np.multiply(np.dot(W2.T,dZ2),(1-(A1)**2));
  dW1=1/m*np.dot(dZ1,X.T);
  db1=1/m*np.sum(dZ1,axis=1,keepdims=True);
  grads={"dW2":dW2,
         "dW1":dW1,
         "db1":db1,
         "db2":db2
         }
  return grads;

In [0]:
def optimise(X,Y,num_iteration,alpha):
    parameters = initialisation(n_inp, n_h, n_out);
    W1 = parameters['W1'];
    b1 = parameters['b1'];
    W2 = parameters['W2'];
    b2 = parameters['b2'];
    costs=[];
    for i in range(num_iteration):
      A2, cache = forward_prop(X, parameters);
      cost = compute_cost(A2, Y);
      costs.append(cost);
      grads = backward_prop(parameters, cache, X, Y);
      dW1=grads['dW1'];
      db1=grads['db1'];
      dW2=grads['dW2'];
      db2=grads['db2'];
      W1 =  W1-alpha * dW1;
      b1 =  b1-alpha * db1;
      W2 =  W2-alpha * dW2;
      b2 =  b2-alpha * db2;
      parameters={"W1": W1,
                "b1":b1,
                "W2":W2,
                "b2":b2,
               }
      
  
   
    return parameters;

In [0]:
def predict(X,parameters):
  A2,cache=forward_prop(X,parameters);
  P=np.round(A2);
  return P;

In [0]:
def model(X_train,Y_train,X_test,Y_test,num_iterations,alpha):
  parameters=optimise(X_train,Y_train,num_iterations,alpha);
  Y_P_train=predict(X_train,parameters);
  Y_P_test=predict(X_test,parameters);
  print(100 - np.mean(np.abs(Y_P_train - Y_train)) * 100);
  print((100 - np.mean(np.abs(Y_P_test - Y_test)) * 100));
  W1 = parameters['W1'];
  W2 = parameters['W2'];
  b1 = parameters['b1'];
  b2 = parameters['b2']; 
  d = {  "Y_prediction_test": Y_P_test, 
         "Y_prediction_train" : Y_P_train, 
         "W1" : W1, 
         "b1" : b1,
         "W2":  W2,
         "b2":  b2,
         "learning_rate" : alpha,
         "num_iterations": num_iterations};
  return d;
  

In [150]:
d = model(X_train.T, Y_train.T, X_test.T ,Y_test.T, num_iterations = 1000, alpha=0.80)

100.0
100.0
